In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
from google.colab import drive
import os

# 1. Mounting Google Drive: This allows Colab to access files in your Google Drive
drive.mount('/content/drive')
basedir = "/content/drive/MyDrive/cs6365/"

In [ ]:
df21 = pd.read_csv('/content/drive/MyDrive/cs6365/merged_gw_2021_22.csv')
df22 = pd.read_csv('/content/drive/MyDrive/cs6365/merged_gw_2022_23.csv')
df23 = pd.read_csv('/content/drive/MyDrive/cs6365/merged_gw_2023_24.csv')

In [ ]:
df23.columns

Columns to drop: Name, team, position, element, fixture, kickoff time (tbd), opponent team, round, transfer_balance


Target data: total_points of next gameweek

preprocessing steps:
- Change was_home to 0/1

# Quick and dirty baseline model

In [ ]:
# Dropping unnecessary columns
train_df = df21.drop(["name", "team", "position", "fixture", "kickoff_time", "opponent_team", "round", "transfers_balance"], axis=1)

# Create a column for ground truth
ground_truth = df21.loc[:, ["element", "total_points", "GW"]]

# total points for GW 2 becomes the ground truth for GW 1
ground_truth['GW'] -= 1
ground_truth = ground_truth.rename(columns={"total_points":"y"})

# inner join with train df
train_df = train_df.merge(ground_truth, how="inner", on=["element", "GW"])

# Dropping 'element' and 'gw' columns as they are no longer needed
train_df = train_df.drop(["element", "GW"], axis=1)
train_df

In [ ]:
#Pre-processing
train_df = train_df.astype(np.float32)
train_df

In [ ]:
# Train model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
class FplDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return torch.tensor(self.inputs.loc[idx], dtype=torch.float32), torch.tensor(self.targets.loc[idx], dtype=torch.float32)

# Convert data to PyTorch Dataset and DataLoader
inputs = train_df.drop('y', axis=1)
num_features = inputs.shape[1]
targets = train_df.loc[:,"y"]
dataset = FplDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self, num_features):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Linear(num_features, 64)  # 64 output units
        self.layer2 = nn.Linear(64, 1)   # 64 input units, 1 output value

    def forward(self, x):
        x = torch.relu(self.layer1(x))  # Apply ReLU activation after first layer
        x = self.layer2(x)              # Output layer
        return x

# Initialize the model, loss function, and optimizer
model = SimpleModel(num_features)
criterion = nn.MSELoss()  # Mean Squared Error loss for regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs_batch, targets_batch in dataloader:
        # Forward pass
        outputs = model(inputs_batch)
        loss = criterion(outputs, targets_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'drive/MyDrive/cs6365/baseline.pth')